In [1]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)
import time
import warnings
warnings.filterwarnings("ignore")

import torch
random_seed = 42
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# LOAD Dataset
from exp.data_loader import load_image_data

dataset_name = 'MNIST'

X_train, X_test, Y_train, Y_test, _, _ = load_image_data(dataset_name,'.data/')

In [3]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)
Y_train = np.eye(10)[Y_train]
Y_test = np.eye(10)[Y_test]

In [4]:
# load Black Box

#NN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
BATCH_SIZE = 1024
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(2048).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(BATCH_SIZE)

##model building
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax'),
])

early_stopping = EarlyStopping(patience=3)
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])
#history = model.fit(
#    train_dataset,
#    validation_data=test_dataset,
#    epochs=100,
#    callbacks=[early_stopping],
#    verbose=2
#)
#def plot_metric(history, metric):
#    train_metrics = history.history[metric]
#    val_metrics = history.history['val_'+metric]
#    epochs = range(1, len(train_metrics) + 1)
#    plt.plot(epochs, train_metrics)
#    plt.plot(epochs, val_metrics)
#    plt.title('Training and validation '+ metric)
#    plt.xlabel("Epochs")
#    plt.ylabel(metric)
#    plt.grid()
#    plt.legend(["train_"+metric, 'val_'+metric])
#    plt.show()
#plot_metric(history, 'loss')
#model.save_weights(f'./blackboxes/{dataset_name}_tf_nn')
from sklearn.metrics import accuracy_score
model.load_weights(f'./blackboxes/{dataset_name}_tf_nn')
model.trainable = False
print(accuracy_score(np.argmax(model.predict(X_train),axis=1),np.argmax(Y_train,axis=1)))
print(accuracy_score(np.argmax(model.predict(X_test),axis=1),np.argmax(Y_test,axis=1)))

2022-08-03 14:38:05.751103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-03 14:38:06.157833: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


0.9973166666666666
0.9911


In [5]:
black_box = 'nn'
def predict(x, return_proba=False):
    if return_proba:
        return model.predict(x)
    else: return np.argmax(model.predict(x),axis=1).astype(int).ravel()
y_test_pred = predict(X_test, return_proba=True)
y_train_pred = predict(X_train, return_proba=True)

In [7]:
from alibi.prototypes import ProtoSelect
from alibi.utils.kernel import EuclideanDistance

summariser = ProtoSelect(kernel_distance=EuclideanDistance(), eps=0.1)

In [10]:
summariser = summariser.fit(X=X_train, y=np.where(Y_train==1)[1])

array([5, 0, 4, ..., 5, 6, 8])